In [1]:
import re


####################### ####################
def get_text(filepath):
    """
        get the text from input file
    """
    text = ""
    with open(filepath, "r", encoding ='utf-8') as inputfile:
        for line in inputfile:
            text = text + line
    return text


def cut_top(text):
    """
        the text before the seller information is not necessary
    """
    begin = 0
    begin_keyword = "From:|Từ:"
#     check_regex = r"(From:|Từ:).+[@].+(Date:|Ngày:)"
#     check_regex = r"(From:|Từ:).*(Date:|Ngày:).*(To:)"
    check_regex = r"(From:|Từ:)[\S\s]+(Date:|Ngày:)[\S\s]+(To:)"
    
    tmp = 0
    while True:
        # Find the last "From" to get the exact location of seller information
        from_found = re.search(begin_keyword, text[tmp:], re.IGNORECASE)
        if from_found and re.match(check_regex, text[tmp+from_found.start():], re.IGNORECASE):
            begin = tmp + from_found.start()
            tmp = tmp + from_found.end()
        else:
            break
               
    return text[begin:]
        


def truncate(text):
    """
        get rid of the unnecessary text
    """
    result = text
    result = cut_top(result)
    return result

In [2]:
###################### GET DATE #####################
def preprocess_and_split_datestring(datestring):
    result = []
    datestring = datestring.replace(",", "")
    result = datestring.split()
    return result


def is_eng_date(datestring):
    month_dict = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for month in month_dict:
        if month in datestring:
            return True
        
    return False


def is_vie_date(datestring):
    pass


def extract_eng_date(datestring):
    """
        get the day, month, year in the english datestring
    """
        
    month_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', \
                  'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
    tokens = preprocess_and_split_datestring(datestring)
    
    year = ''
    month = ''
    day = ''
    
    yearregex = r'(19|20)\d{2}'
    dayregex = r'([12]\d|3[01]|0?[1-9])'
    
    for token in tokens:
        """
            loop over all the integer in the string to check if it is day or month or year
        """
        if (month=='') and (token in month_dict):
            month = month_dict[token]
        elif (day=='') and re.match(dayregex, token):
            day = token
        elif (year=='') and re.match(yearregex, token):
            year = token
            
    if len(day)==1:
        day = '0' + day
    return day + '/' + month + '/' + year


def extract_vie_date(datestring):
    """
        get the day, month, year in the english datestring
    """
    substring_found = re.search(r",.+,.+", datestring)
    if substring_found:
        datestring = substring_found.group()
    else:
        return ""
    
    tokens = preprocess_and_split_datestring(datestring)
    
    year = ''
    month = ''
    day = ''
    
    yearregex = r'(19|20)\d{2}'
    monthregex = r'(1[012]|0?[1-9])'
    dayregex = r'([12]\d|3[01]|0?[1-9])'
    
    for token in tokens:
        """
            loop over all the integer in the string to check if it is day or month or year
        """
        if (day=='') and re.match(dayregex, token):
            day = token
        elif (month=='') and re.match(monthregex, token):
            month = token
        elif (year=='') and re.match(yearregex, token):
            year = token
            
    if len(day)==1:
        day = '0' + day
    if len(month)==1:
        month = '0' + month
    return day + '/' + month + '/' + year


def extract_date(datestring):
    if is_eng_date(datestring):
#         print("eng")
        return extract_eng_date(datestring)
    else:
#         print("vie")
        return extract_vie_date(datestring)
    

def get_date(text):
    """
        Get date from the text
    """
    FIELDS = ['From|Từ', 'Subject|Chủ đề', 'Date|Ngày', 'To|Tới']
    key_field = 'Date|Ngày'
    
    begin = 0
    end = len(text)
    
    found = re.search(key_field, text[begin:end], re.IGNORECASE)
    if found:
        # search for the key_field (begin point of result)
        begin = begin + found.end()
        if text[begin]==':':
            begin += 1
            
        for field in FIELDS:
            # search for the other fields (end point of result)
            endfound = re.search(field, text[begin:end], re.IGNORECASE)
            if endfound:
                # update the endpoint (the closest field to the key_field)
                end = min(end, begin + endfound.start())
    
    return extract_date(text[begin:end].strip())

In [3]:
def extract_from_email_bodytext(text):
    """
        from body text of the email, extract the information
    """   
    result = {}
    text = truncate(text)
    
    result.update({'date': get_date(text)})
    
    return result

In [4]:
inputfile = "bodytext/36.txt"
text = get_text(inputfile)
# print(text)
text = truncate(text)

print(extract_from_email_bodytext(text))

{'date': '02/07/2019'}


In [7]:
import Beta
n_test = 47
for i in range(43, n_test+1):
    inputfile = "bodytext/" + str(i) + ".txt"
    text = get_text(inputfile)
    print(Beta.extract_from_email_bodytext(text))

{'created_by': 'laclac110897@gmail.com', 'date': '18/11/2019', 'seller': '* Tiki', 'total': 809100.0, 'currency': 'VND'}
{'created_by': 'laclac110897@gmail.com', 'date': '12/07/2019', 'seller': '* Tiki', 'total': 99000.0, 'currency': 'VND'}
{'created_by': 'laclac110897@gmail.com', 'date': '08/09/2019', 'seller': 'MoMo', 'total': 100000, 'currency': 'VND'}
{'created_by': 'laclac110897@gmail.com', 'date': '03/09/2019', 'seller': 'MoMo', 'total': 65700, 'currency': 'VND'}
{'created_by': 'laclac110897@gmail.comq', 'date': '02/10/2019', 'seller': 'be', 'total': 10000, 'currency': 'VND'}
